## Notebook: Transform dữ liệu từ tầng broze lên tầng silver

### 1: Import & Connect DB

In [ ]:
import os
import json
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import BigInteger, VARCHAR, DateTime, Text
from dotenv import load_dotenv

# Load biến môi trường từ file .env
load_dotenv()

API_KEY   = os.getenv("API_KEY")
DB_USER   = os.getenv("DB_USER")
DB_PASS   = os.getenv("DB_PASS")
DB_HOST   = os.getenv("DB_HOST")
DB_PORT   = os.getenv("DB_PORT")
DB_BRONZE = os.getenv("DB_BRONZE")
DB_SILVER = os.getenv("DB_SILVER")

# Tạo engine kết nối đến Bronze DB và Silver DB
bronze_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_BRONZE}")
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")

### 2. Load dữ liệu từ Bronze & Liệt kê keys JSON (Products)

In [ ]:
products_df = pd.read_sql("SELECT * FROM products_raw", bronze_engine)
print("Shape:", products_df.shape)

# Liệt kê keys cấp 1
all_keys = set()
for raw in products_df["raw_json"].head(200):
    d = json.loads(raw)
    all_keys.update(d.keys())

print("===== Tổng số keys:", len(all_keys), " =====")
print(sorted(all_keys))

# Hiển thị 1 record mẫu
print(json.dumps(json.loads(products_df["raw_json"].iloc[0]), indent=2, ensure_ascii=False))


Shape: (37, 4)
===== Tổng số keys: 47  =====
['bonus_products', 'brand_id', 'categories', 'category_ids', 'creator_id', 'custom_id', 'description', 'display_id', 'display_id_original', 'duration', 'has_composite', 'hidden_fb_catalog_ids', 'hide_product_name', 'id', 'ignore_awarded_point', 'image', 'inserted_at', 'is_hide', 'is_price_by_weight', 'is_published', 'is_sell_negative', 'keyword', 'lifecycle_days', 'limit_quantity_to_warn', 'manipulation_warehouses', 'material_names', 'materials', 'measure_group_id', 'name', 'not_print_product', 'note', 'note_product', 'offline_id', 'product_attributes', 'product_links', 'removed', 'shop_id', 'shop_warranty_policy_id', 'store_links', 'supplier_product_ids', 'tags', 'third_parties', 'type', 'updated_at', 'variations', 'warning_by_variation', 'warranty_period']
{
  "categories": [
    {
      "id": 1290005112,
      "name": "Tam Anh"
    }
  ],
  "lifecycle_days": 0,
  "note_product": "",
  "name": "Áo thu đông.W033",
  "type": "product",
  "di

### 3. Parse dim_products

In [ ]:

def parse_product(row):
    d = json.loads(row["raw_json"])
    return {
        "product_id": d.get("id"),
        "shop_id": d.get("shop_id"),
        "name": d.get("name"),
        "custom_id": d.get("custom_id"),
        "brand_id": d.get("brand_id"),
        "category_ids": ",".join(map(str, d.get("category_ids", []))) if d.get("category_ids") else None,
        "description": d.get("description"),
        "inserted_at": d.get("inserted_at"),
        "updated_at": d.get("updated_at")
    }

dim_products = products_df.apply(parse_product, axis=1, result_type="expand").drop_duplicates("product_id")
print("✅ dim_products shape:", dim_products.shape)


✅ dim_products shape: (37, 9)


### 4. Hàm Data Quality Check

In [ ]:

def data_quality_report(df, pk=None, sample_n=3):
    report = []
    for col in df.columns:
        null_pct = df[col].isna().mean() * 100
        unique_cnt = df[col].nunique(dropna=True)
        sample_vals = df[col].dropna().unique()[:sample_n]

        report.append({
            "column": col,
            "dtype": str(df[col].dtype),
            "null_%": round(null_pct, 2),
            "unique_count": unique_cnt,
            "sample_values": sample_vals
        })
    report_df = pd.DataFrame(report)

    if pk:
        dup_cnt = df.duplicated(subset=pk).sum()
        print(f"🔍 Duplicate check on {pk}: {dup_cnt} duplicate rows found")

    return report_df



In [13]:
dq_products = data_quality_report(dim_products, pk=["product_id"])
display(dq_products)

# Xem nhanh tỷ lệ null từng cột
null_rate = dim_products.isna().mean().sort_values(ascending=False)
print("\n🔎 Null percentage by column:")
print(null_rate)


🔍 Duplicate check on ['product_id']: 0 duplicate rows found


,column,dtype,null_%,unique_count,sample_values
0,product_id,object,0.00,37,"[3ec3103b-ee8e-484a-abf3-d542f4487c18, 6baccc2..."
1,shop_id,int64,0.00,1,[230361475]
2,name,object,0.00,37,"[Áo thu đông.W033, Áo Khoác.W032, Áo thu đông...."
3,custom_id,object,0.00,37,"[WINNER - Áo thu đông.W033, WINNER - Áo Khoác...."
4,brand_id,object,100.00,0,[]
5,category_ids,object,13.51,9,"[1290005112, 1290028415, 1290021501]"
6,description,object,100.00,0,[]
7,inserted_at,object,0.00,37,"[2024-12-23T08:36:41, 2024-12-23T06:00:26, 202..."
8,updated_at,object,0.00,36,"[2024-12-23T08:36:41, 2024-12-25T03:13:05, 202..."



🔎 Null percentage by column:
brand_id        1.000000
description     1.000000
category_ids    0.135135
product_id      0.000000
shop_id         0.000000
custom_id       0.000000
name            0.000000
inserted_at     0.000000
updated_at      0.000000
dtype: float64


### 5. Drop high-null columns (dim_products)

In [ ]:

def drop_high_nulls(df, threshold=0.7):
    """
    Xóa cột có tỷ lệ null > threshold
    """
    null_pct = df.isna().mean()
    drop_cols = null_pct[null_pct > threshold].index.tolist()
    
    if drop_cols:
        print(f"⚠️ Dropping {len(drop_cols)} columns (> {threshold*100:.0f}% null): {drop_cols}")
        return df.drop(columns=drop_cols)
    else:
        print("✅ No columns dropped (all under threshold)")
        return df

dim_products_clean = drop_high_nulls(dim_products, threshold=0.7)
print("✅ dim_products_clean shape:", dim_products_clean.shape)


⚠️ Dropping 2 columns (> 70% null): ['brand_id', 'description']
✅ dim_products_clean shape: (37, 7)


In [15]:
dim_products_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_id    37 non-null     object
 1   shop_id       37 non-null     int64 
 2   name          37 non-null     object
 3   custom_id     37 non-null     object
 4   category_ids  32 non-null     object
 5   inserted_at   37 non-null     object
 6   updated_at    37 non-null     object
dtypes: int64(1), object(6)
memory usage: 2.2+ KB


In [ ]:
# Chuyển đổi kiểu dữ liệu
dim_products_clean["inserted_at"] = pd.to_datetime(dim_products_clean["inserted_at"], errors="coerce")
dim_products_clean["updated_at"] = pd.to_datetime(dim_products_clean["updated_at"], errors="coerce")

print(dim_products_clean.dtypes)


product_id              object
shop_id                  int64
name                    object
custom_id               object
category_ids            object
inserted_at     datetime64[ns]
updated_at      datetime64[ns]
dtype: object


### 6. Load vào Silver

In [ ]:
dtype_dim_products = {
    "product_id": VARCHAR(100),
    "shop_id": BigInteger(),
    "name": VARCHAR(255),
    "custom_id": VARCHAR(255),
    "category_ids": Text(),   # vì có thể chứa nhiều ID join sang dim_categories
    "inserted_at": DateTime(),
    "updated_at": DateTime()
}

dim_products_clean.to_sql(
    "dim_products",
    silver_engine,
    if_exists="replace",
    index=False,
    dtype=dtype_dim_products
)

print("🚀 Loaded dim_products → Silver DB")


🚀 Loaded dim_products → Silver DB
